In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import os
import nltk
import re
import heapq
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation,Dropout
from keras.layers import LSTM
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
import pickle
import matplotlib.pyplot as plt



In [2]:
jumia_data=pd.read_csv("data_sets\jumia_clothes\jumia_clothes.csv")
myntra_data=pd.read_csv("data_sets\myntra_cloth\myntra_products_catalog.csv")

In [3]:
jumia_data.head()

,id,name,brand,category,price$,boutique_officielle,price,gender,price_before,promotion,reviews,numberofReviewers,Livraison_gratuite,Date
0,GE232FA1FQVNSNAFAMZ,Hoodie/Capuchon Death Note Anime - Otaku,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,6.04,No,64.60 Dhs,Homme,233.00 Dhs,72%,3.2 out of 5,(81),No,2022-03-25
1,GE232MW1JMM00NAFAMZ,Hoodie Manga Naruto Anime Itachi Uchiha Sweats...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,13.92,No,149.00 Dhs,Homme,192.00 Dhs,22%,4 out of 5,(11),No,2022-03-25
2,GE232MW011137NAFAMZ,Hoodie Manga Anime Naruto Shippuden Sweatshirt...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,20.46,No,219.00 Dhs,Homme,260.00 Dhs,16%,4.5 out of 5,(19),No,2022-03-25
3,GE232FA1MGS7TNAFAMZ,Hoodie / Capuchon noir avec motif de ATTACK ON...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,7.90,No,84.55 Dhs,Homme,399.00 Dhs,79%,3.4 out of 5,(35),No,2022-03-25
4,GE232MW0X9UR9NAFAMZ,Hoodie Manga Anime Naruto Shippuden Sweatshirt...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,13.92,No,149.00 Dhs,Homme,285.00 Dhs,48%,3.3 out of 5,(33),No,2022-03-25


In [4]:
myntra_data.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [5]:
def clean_data(dataset,col_name):
    
    dataset=dataset[col_name].apply(lambda x: x.lower())
    dataset=dataset.apply(lambda x: x.replace(u'/',u' '))
    dataset=dataset.apply(lambda x: x.replace(u'&',u'and'))
    #dataset=dataset.apply(lambda x: x.replace(u'-',u' '))
    dataset=dataset.apply(lambda x: x.replace(u'_',u' '))
    dataset=dataset.apply(lambda x: x.replace(u'\n',u' '))
    dataset=dataset.apply(lambda x: re.sub(' +', ' ', x))
    return dataset

In [6]:
myntra_clean=clean_data(myntra_data,'ProductName')
myntra_clean.head()

0    dkny unisex black and grey printed medium trol...
1    ethnovogue women beige and grey made to measur...
2    spykar women pink alexa super skinny fit high-...
3    raymond men blue self-design single-breasted b...
4    parx men brown and off-white slim fit printed ...
Name: ProductName, dtype: object

In [7]:
jumia_clean=clean_data(jumia_data,'name')
jumia_clean.head()

0             hoodie capuchon death note anime - otaku
1    hoodie manga naruto anime itachi uchiha sweats...
2    hoodie manga anime naruto shippuden sweatshirt...
3    hoodie capuchon noir avec motif de attack on t...
4    hoodie manga anime naruto shippuden sweatshirt...
Name: name, dtype: object

In [8]:
def remove_nonenglish_data(dataset):
    rows=[]
    x=[]
    #jumia_try=jumia_data
    words = set(nltk.corpus.words.words())
    from tqdm import tqdm

    for i in tqdm(range(len(dataset))):
        for w  in nltk.word_tokenize(dataset[i]):
            if w not in words:
                rows.append(i)
                break
    english_data=dataset.drop(rows)
    return english_data

In [9]:
jumia_clean=remove_nonenglish_data(jumia_clean)



100%|██████████████████████████████████████████████████████████████████████████| 45138/45138 [00:05<00:00, 7551.64it/s]


In [10]:
print(jumia_clean[1500:1520])

Series([], Name: name, dtype: object)


In [11]:
print(len(jumia_clean))
jumia_clean=jumia_clean.drop_duplicates()
print(len(jumia_clean))


1246
545


In [12]:
print(len(myntra_clean))
myntra_clean=myntra_clean.drop_duplicates()
print(len(myntra_clean))

12491
10749


In [13]:
data= pd.concat([myntra_clean,jumia_clean]).drop_duplicates(keep=False)
len(data)

11294

In [14]:
data.head()

0    dkny unisex black and grey printed medium trol...
1    ethnovogue women beige and grey made to measur...
2    spykar women pink alexa super skinny fit high-...
3    raymond men blue self-design single-breasted b...
4    parx men brown and off-white slim fit printed ...
dtype: object

In [21]:
data=shuffle(data)

In [22]:
data.head()

11852      jompers women green woven design straight kurta
9940                maniac men charcoal grey solid joggers
3204     obsessions brown and grey colourblocked poly a...
12442      miss chase women floral print purple maxi dress
1729     xyxx men intellisoft antimicrobial micro modal...
dtype: object

In [23]:
text=' '.join(data)
print('length of the corpus is: :', len(text))


length of the corpus is: : 612748


In [24]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)


In [25]:
words[60:70]

['terrain',
 'men',
 'pink',
 'and',
 'navy',
 'blue',
 'chiseled',
 'slim',
 'fit',
 'checked']

In [26]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [27]:
len(unique_words)
unique_words[0:10]

array(['0', '001', '002', '004', '01', '010', '015', '017v', '01b', '02'],
      dtype='<U18')

In [33]:
def n_grams (N,words):

    n_grams_prev_words = []
    n_grams_next_words = []
    for j in range(len(words) - N):
        n_grams_prev_words.append(words[j:j + N])
        n_grams_next_words.append(words[j + N])
    return n_grams_prev_words,n_grams_next_words

In [34]:
N=5
prev_words,next_word=n_grams(N,words)

In [35]:
print (prev_words[0],"   ",next_word[0])

['next', 'look', 'men', 'maroon', 'slim']     fit


In [36]:
#word2vec
def word2vec(prev_words,next_word):
    X = np.zeros((len(prev_words), N, len(unique_words)), dtype=bool)
    Y = np.zeros((len(next_word), len(unique_words)), dtype=bool)

    for i, each_words in enumerate(prev_words):
        for j, each_word in enumerate(each_words):
            X[i, j, unique_word_index[each_word]] = 1
        Y[i, unique_word_index[next_word[i]]] = 1
    return unique_word_index,X,Y

In [37]:
word2id,X,Y=word2vec(prev_words,next_word)

In [38]:
word2id #126black': 74

{'0': 0,
 '001': 1,
 '002': 2,
 '004': 3,
 '01': 4,
 '010': 5,
 '015': 6,
 '017v': 7,
 '01b': 8,
 '02': 9,
 '02063': 10,
 '027': 11,
 '02a': 12,
 '03': 13,
 '030': 14,
 '04': 15,
 '046': 16,
 '047': 17,
 '049': 18,
 '05': 19,
 '050': 20,
 '052': 21,
 '053': 22,
 '054': 23,
 '055': 24,
 '056': 25,
 '058': 26,
 '059': 27,
 '06': 28,
 '060': 29,
 '061': 30,
 '08': 31,
 '0830669': 32,
 '08gms': 33,
 '09': 34,
 '1': 35,
 '10': 36,
 '100': 37,
 '1000': 38,
 '100g': 39,
 '100gm': 40,
 '100ml': 41,
 '1019mg2752': 42,
 '1019mg2753': 43,
 '10248015': 44,
 '10248017': 45,
 '10248023': 46,
 '10248037': 47,
 '105': 48,
 '109f': 49,
 '10g': 50,
 '11': 51,
 '110': 52,
 '110ml': 53,
 '1114': 54,
 '1170': 55,
 '12': 56,
 '120': 57,
 '120apricot': 58,
 '120blue': 59,
 '120bright': 60,
 '120peach': 61,
 '121pj': 62,
 '1228': 63,
 '1229': 64,
 '1230': 65,
 '1234': 66,
 '1235': 67,
 '125': 68,
 '125002': 69,
 '125201': 70,
 '125601': 71,
 '125701': 72,
 '125g': 73,
 '126black': 74,
 '126ivory': 75,
 '1290b

In [39]:
def id2word(word2id):
    return {y:x for x,y in word2id.items()}

In [40]:
id2word = {y:x for x,y in word2id.items()}
id2word

{0: '0',
 1: '001',
 2: '002',
 3: '004',
 4: '01',
 5: '010',
 6: '015',
 7: '017v',
 8: '01b',
 9: '02',
 10: '02063',
 11: '027',
 12: '02a',
 13: '03',
 14: '030',
 15: '04',
 16: '046',
 17: '047',
 18: '049',
 19: '05',
 20: '050',
 21: '052',
 22: '053',
 23: '054',
 24: '055',
 25: '056',
 26: '058',
 27: '059',
 28: '06',
 29: '060',
 30: '061',
 31: '08',
 32: '0830669',
 33: '08gms',
 34: '09',
 35: '1',
 36: '10',
 37: '100',
 38: '1000',
 39: '100g',
 40: '100gm',
 41: '100ml',
 42: '1019mg2752',
 43: '1019mg2753',
 44: '10248015',
 45: '10248017',
 46: '10248023',
 47: '10248037',
 48: '105',
 49: '109f',
 50: '10g',
 51: '11',
 52: '110',
 53: '110ml',
 54: '1114',
 55: '1170',
 56: '12',
 57: '120',
 58: '120apricot',
 59: '120blue',
 60: '120bright',
 61: '120peach',
 62: '121pj',
 63: '1228',
 64: '1229',
 65: '1230',
 66: '1234',
 67: '1235',
 68: '125',
 69: '125002',
 70: '125201',
 71: '125601',
 72: '125701',
 73: '125g',
 74: '126black',
 75: '126ivory',
 76: '1

In [41]:
model = Sequential()
model.add(LSTM(128, input_shape=(N, len(unique_words))))
#model.add(Dropout(.2))
model.add(Dense(len(unique_words)))
model.add(Dropout(.2))
model.add(Activation('softmax'))

In [42]:

optimizer = Adam(learning_rate=0.05)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=64, epochs=10, shuffle=True).history

Epoch 1/10
1501/1501 [==============================] - 92s 59ms/step - loss: 4.4821 - accuracy: 0.3390 - val_loss: 3.0295 - val_accuracy: 0.4537
Epoch 2/10
1501/1501 [==============================] - 84s 56ms/step - loss: 3.8760 - accuracy: 0.3928 - val_loss: 2.8984 - val_accuracy: 0.4695
Epoch 3/10
1501/1501 [==============================] - 84s 56ms/step - loss: 3.6642 - accuracy: 0.4078 - val_loss: 2.8405 - val_accuracy: 0.4802
Epoch 4/10
1206/1501 [=======================>......] - ETA: 16s - loss: 3.5276 - accuracy: 0.4212

KeyboardInterrupt: 

In [59]:

optimizer = RMSprop(learning_rate=0.05)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=64, epochs=10, shuffle=True).history

Epoch 1/10
1501/1501 [==============================] - 157s 102ms/step - loss: 5.3425 - accuracy: 0.4206 - val_loss: 5.4508 - val_accuracy: 0.4598
Epoch 2/10
1501/1501 [==============================] - 120s 80ms/step - loss: 5.3181 - accuracy: 0.4243 - val_loss: 5.5038 - val_accuracy: 0.4547
Epoch 3/10
1501/1501 [==============================] - 147s 98ms/step - loss: 5.4356 - accuracy: 0.4187 - val_loss: 5.6182 - val_accuracy: 0.4456
Epoch 4/10
1501/1501 [==============================] - 133s 88ms/step - loss: 5.2637 - accuracy: 0.4241 - val_loss: 5.3842 - val_accuracy: 0.4513
Epoch 5/10
1501/1501 [==============================] - 135s 90ms/step - loss: 5.0720 - accuracy: 0.4290 - val_loss: 5.5341 - val_accuracy: 0.4585
Epoch 6/10
1501/1501 [==============================] - 147s 98ms/step - loss: 5.2033 - accuracy: 0.4252 - val_loss: 5.4611 - val_accuracy: 0.4563
Epoch 7/10
1501/1501 [==============================] - 160s 107ms/step - loss: 5.0334 - accuracy: 0.4313 - val_loss:

In [60]:
model.save('next_word_model.h5')


In [45]:
model = load_model('next_word_model.h5')

In [46]:
pickle.dump(history, open("history.p", "wb"))
history

NameError: name 'history' is not defined

In [47]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

NameError: name 'history' is not defined

In [70]:
def prepare_input(text):
    words = tokenizer.tokenize(text)
    if len(words)>5:
        words = words[-5:]
    x = np.zeros((1, N, len(unique_words)))
    for t, word in enumerate(words):
        if word in word2id:
            x[0, t, word2id[word]] = 1.
    print(words) 
    return x
     

def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)
     

def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_word = id2word[next_index]
        text = text[1:] + next_word
        completion += next_word
        
        if len(original_text + completion) + 2 > len(original_text) and next_word == ' ':
            return completion

        
def predict_next_word(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [id2word[idx] for idx in next_indices]

In [71]:
input="pepe jeans men white"
print(predict_next_word(input, 5))
print(len(input.split()))

['pepe', 'jeans', 'men', 'white']
['slim', 'round', 'printed', 'polo', 'self']
4


C:\Users\Ahmed\AppData\Local\Temp\ipykernel_19500\3476847755.py:15: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds)
